In [3]:
# Importing db libraries
import pandas as pd
import sqlalchemy as sa
import pyodbc
import time
import numpy as np
import json

In [4]:
def get_cnn_value_from_json(json_local):
    json.load(json_local)

def stringfy(classes):
    r = ""
    for i in classes:
        r += str(i)
    return r.strip()

### M3100 procesado

In [5]:
m3100_pro = pd.read_csv('C:\\git\\cuponesWong\\CuponesWong\\notebooks_flow\\local_result\\result_new_train_with_line_full.csv', dtype={'DNI':str, 'Telefono':str})
m3100_pro = m3100_pro.fillna('')

In [6]:
m3100_pro['Azure'] = np.where(
    np.logical_and(
        np.logical_and(m3100_pro['DNI'].apply(len)==8, m3100_pro.AcertividadDNI>=97.0),
        np.logical_and(
            np.logical_and(m3100_pro['Telefono'].apply(len)<=9,m3100_pro['Telefono'].apply(len)>=6),
            m3100_pro.AcertividadTelefono>=97.0
            )
        ),
    0,
    1)

In [18]:
(m3100_pro[m3100_pro['DNI'].apply(len)==8]['idCupon'].values.size) / (3157)

0.6126069052898321

### M3100 real

In [7]:
m3100_real = pd.read_csv('D:\\GyS\\proyectos\\cuponesWong\\proceso_definitivo\\process\\muestra900\\levantamiento\\total.csv', dtype={'DNI':str, 'telefono':str})
m3100_real = m3100_real.fillna('')
m3100_real['NombreArchivo'] = m3100_real['nombre_archivo'].apply(lambda x: x[:-4])
m3100_real.head()

,Unnamed: 0,nombre_archivo,DNI,telefono,Observación,email,NombreArchivo
0,0,MarzoCompras_0202072019164448_00043.jpg,42650934,979754672,,Maicusi@hotmail.com,MarzoCompras_0202072019164448_00043
1,1,MarzoCompras_0202072019164454_00245.jpg,20996780,923540008,,,MarzoCompras_0202072019164454_00245
2,2,MarzoCompras_0202072019164458_00359.jpg,42051024,940445230,,jtrujillowong@outlook.com,MarzoCompras_0202072019164458_00359
3,3,MarzoCompras_0202072019164503_00537.jpg,40557762,987136744,,Micasu -80@Hotmail.com,MarzoCompras_0202072019164503_00537
4,4,MarzoCompras_0202072019164503_00546.jpg,08668202,995382267,,CAROLINA CM10_10@HOTMAIL.COM,MarzoCompras_0202072019164503_00546


In [8]:
total = pd.merge(m3100_pro, m3100_real, how='left', on='NombreArchivo')
# total['dni_yolo'] = total['LocalJsonOCR'].apply(lambda x: stringfy(json.loads(x)['dni_yolo']['values']))
# total['tel_yolo'] = total['LocalJsonOCR'].apply(lambda x: stringfy(json.loads(x)['telefono_yolo']['values']))
# total['acert_dni_mean'] = total['LocalJsonOCR'].apply(lambda x: np.array(json.loads(x)['dni_yolo']['scores']).sum()/(np.array(json.loads(x)['dni_yolo']['scores']).size * 100))
# total['acerts_dni'] = total['LocalJsonOCR'].apply(lambda x: json.loads(x)['dni_yolo']['scores'])
total.head()

,NombreArchivo,idCupon,DNI_x,AcertividadDNI,Telefono,AcertividadTelefono,idCampania,idUsuario,idEstado,LocalJsonOCR,Azure,Unnamed: 0,nombre_archivo,DNI_y,telefono,Observación,email
0,MarzoCompras-0303072019121007_00008,1000,600672266,0.00,08732143,93.30,4,1,1,"{""dni_cnn"": {""value"": ""600672266""}, ""telefono_...",1,2097,MarzoCompras-0303072019121007_00008.jpg,000672966,982331400,,karaujo71@hotmail.com
1,MarzoCompras-0303072019121007_00010,1001,49881016,96.30,2946428522,0.00,4,1,1,"{""dni_cnn"": {""value"": ""49881016""}, ""telefono_c...",1,889,MarzoCompras-0303072019121007_00010.jpg,09888016,946498597,,Mauricio280156@Hotmail.com
2,MarzoCompras-0303072019121007_00025,1002,04301500,96.18,4282182114,0.00,4,1,1,"{""dni_cnn"": {""value"": ""04301500""}, ""telefono_c...",1,1193,MarzoCompras-0303072019121007_00025.jpg,04015880,928489578,,-
3,MarzoCompras-0303072019121013_00265,1003,04310242,94.58,905973705,99.26,4,1,1,"{""dni_cnn"": {""value"": ""04310242""}, ""telefono_c...",1,890,MarzoCompras-0303072019121013_00265.jpg,09310742,985973705,,Jalina_9@yahoo.es
4,MarzoCompras-0303072019121014_00331,1004,0431060,0.00,9575602,89.67,4,1,1,"{""dni_cnn"": {""value"": ""0431060""}, ""telefono_cn...",1,585,MarzoCompras-0303072019121014_00331.jpg,08281360,949357892,,


# Lev DNI

In [11]:
total_dni = total[['NombreArchivo','DNI_x','DNI_y','AcertividadDNI']]
# total_dni['dni_lev'] = np_distance(total_dni['dni_yolo'],total_dni['DNI_y'])
# total_dni['dni_conj'] = np_improve(total_dni['dni_yolo'],total_dni['DNI_x'],total_dni['acerts_dni'])
# total_dni['dni_lev_conj'] = np_distance(total_dni['dni_conj'],total_dni['DNI_y'])
total_dni['dni_lev'] = np_distance(total_dni['DNI_x'],total_dni['DNI_y'])
total_dni.head()

C:\Users\GYS\AppData\Local\conda\conda\envs\tfkeras\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,NombreArchivo,DNI_x,DNI_y,AcertividadDNI,dni_lev
0,MarzoCompras-0303072019121007_00008,600672266,000672966,0.00,2
1,MarzoCompras-0303072019121007_00010,49881016,09888016,96.30,2
2,MarzoCompras-0303072019121007_00025,04301500,04015880,96.18,3
3,MarzoCompras-0303072019121013_00265,04310242,09310742,94.58,2
4,MarzoCompras-0303072019121014_00331,0431060,08281360,0.00,4


In [12]:
lev_dni = total_dni.groupby(['dni_lev']).agg({'NombreArchivo':'count'})
lev_dni['Porcentaje'] = np.round(lev_dni['NombreArchivo'] * 100 / lev_dni['NombreArchivo'].sum() , 2 )
lev_dni['Cuenta'] = lev_dni['NombreArchivo']
lev_dni[['Cuenta','Porcentaje']]

,Cuenta,Porcentaje
dni_lev,,
0,379,12.00
1,679,21.49
2,786,24.88
3,590,18.68
4,366,11.59
5,191,6.05
6,114,3.61
7,33,1.04
8,16,0.51


In [102]:
lev_dni_conj = total_dni.groupby(['dni_lev_conj']).agg({'NombreArchivo':'count'})
lev_dni_conj['perc'] = lev_dni_conj['NombreArchivo'] * 100 / lev_dni_conj['NombreArchivo'].sum()
lev_dni_conj

,NombreArchivo,perc
dni_lev_conj,,
0,406,12.852168
1,747,23.646724
2,748,23.678379
3,556,17.600506
4,357,11.301045
5,210,6.647673
6,84,2.659069
7,36,1.139601
8,12,0.379867


In [115]:
lev_dni_cnn = total_dni.groupby(['dni_lev_cnn']).agg({'NombreArchivo':'count'})
lev_dni_cnn['perc'] = lev_dni_cnn['NombreArchivo'] * 100 / lev_dni_cnn['NombreArchivo'].sum()
lev_dni_cnn

,NombreArchivo,perc
dni_lev_cnn,,
0,406,12.852168
1,747,23.646724
2,748,23.678379
3,556,17.600506
4,357,11.301045
5,210,6.647673
6,84,2.659069
7,36,1.139601
8,12,0.379867


# Lev Telefono

In [22]:
total_tel = total[['NombreArchivo','Telefono','telefono','AcertividadTelefono','tel_yolo']]
total_tel['tel_lev'] = np_distance(total_tel['Telefono'],total_tel['telefono'])
total_tel['tel_lev_yolo'] = np_distance(total_tel['tel_yolo'],total_tel['telefono'])
total_tel.head()

C:\Users\GYS\AppData\Local\conda\conda\envs\tfkeras\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\GYS\AppData\Local\conda\conda\envs\tfkeras\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,NombreArchivo,Telefono,telefono,AcertividadTelefono,tel_yolo,tel_lev,tel_lev_yolo
0,MarzoCompras-0303072019121007_00008,40736,982331400,0.00,70720,8,8
1,MarzoCompras-0303072019121007_00010,7469851,946498597,71.26,4467852,4,5
2,MarzoCompras-0303072019121007_00025,92070524,928489578,61.44,92070077,6,6
3,MarzoCompras-0303072019121013_00265,90023702,985973705,54.40,90023705,5,4
4,MarzoCompras-0303072019121014_00331,7459002,949357892,82.79,7057002,6,6


In [23]:
lev_tel = total_tel.groupby(['tel_lev']).agg({'NombreArchivo':'count'})
lev_tel['Porcentaje'] = np.round(lev_tel['NombreArchivo'] * 100 / lev_tel['NombreArchivo'].sum() , 2 )
lev_tel['Cuenta'] = lev_tel['NombreArchivo']
lev_tel[['Cuenta','Porcentaje']]

,Cuenta,Porcentaje
tel_lev,,
0,120,3.80
1,356,11.27
2,549,17.38
3,647,20.48
4,643,20.35
5,429,13.58
6,235,7.44
7,110,3.48
8,51,1.61


In [24]:
lev_tel_yolo = total_tel.groupby(['tel_lev_yolo']).agg({'NombreArchivo':'count'})
lev_tel_yolo['Porcentaje'] = np.round(lev_tel_yolo['NombreArchivo'] * 100 / lev_tel_yolo['NombreArchivo'].sum() , 2 )
lev_tel_yolo['Cuenta'] = lev_tel_yolo['NombreArchivo']
lev_tel_yolo[['Cuenta','Porcentaje']]

,Cuenta,Porcentaje
tel_lev_yolo,,
0,61,1.93
1,191,6.05
2,374,11.84
3,597,18.90
4,676,21.40
5,569,18.01
6,389,12.31
7,194,6.14
8,81,2.56


In [10]:
def aciertos_from_str(real, est):
    count = 0
    for i in range(len(real)):
        if real[i]==est[i]:
            count += 1
    return count

def distance(str1, str2):
    d=dict()
    for i in range(len(str1)+1):
        d[i]=dict()
        d[i][0]=i
    for i in range(len(str2)+1):
        d[0][i] = i
    for i in range(1, len(str1)+1):
        for j in range(1, len(str2)+1):
            d[i][j] = min(d[i][j-1]+1, d[i-1][j]+1, d[i-1][j-1]+(not str1[i-1] == str2[j-1]))
    return d[len(str1)][len(str2)]

def improve(num, num_pred, scores):
    res = ""
    for i in range(len(str(num))):
        if scores[i] >= 9400:
            res += str(str(num)[i])
        else:
            res += str(str(num_pred)[i])
    return res

np_aciertos_from_str = np.vectorize(aciertos_from_str)
np_distance = np.vectorize(distance)
np_improve = np.vectorize(improve)

### Levensthein DNI

In [24]:
total_dni = total[['NombreArchivo','DNI_x','dni_yolo','DNI_y','AcertividadDNI','acert_dni_mean','acerts_dni']]
total_dni

,NombreArchivo,DNI_x,dni_yolo,DNI_y,AcertividadDNI,acert_dni_mean,acerts_dni
0,MarzoCompras-0303072019121007_00008,00622926,00672766,000672966,93.76,97.308750,"[9918, 9957, 9851, 9376, 9971, 9601, 9767, 9406]"
1,MarzoCompras-0303072019121007_00010,0588806,0580504,09888016,0.00,77.671429,"[6123, 6384, 9777, 9938, 8264, 7833, 6051]"
2,MarzoCompras-0303072019121007_00025,04015560,04015000,04015880,60.25,88.405000,"[8678, 6025, 9630, 7339, 9627, 9851, 9654, 9920]"
3,MarzoCompras-0303072019121013_00265,031027,031022,09310742,0.00,75.090000,"[7661, 7926, 8730, 9320, 5001, 6416]"
4,MarzoCompras-0303072019121014_00331,082813622,002217602,08281360,0.00,84.056667,"[9902, 9262, 6085, 6364, 9736, 9585, 9761, 820..."
5,MarzoCompras-0303072019121014_00343,00270147,00740147,09740147,91.44,97.861250,"[9856, 9719, 9940, 9807, 9999, 9990, 9834, 9144]"
6,MarzoCompras-0303072019121017_00450,9900157,7900157,09900151,0.00,94.557143,"[8571, 7843, 9996, 9969, 9954, 9886, 9971]"
7,MarzoCompras-0303072019121017_00459,08482229,00482219,08482279,74.83,91.778750,"[9957, 9401, 9468, 7483, 9954, 9463, 8462, 9235]"
8,MarzoCompras-0303072019121018_00497,452775,452774,45279490,0.00,94.658333,"[9945, 9704, 9890, 9936, 9875, 7445]"
9,MarzoCompras-0303072019121018_00501,0533314,0523314,05333314,0.00,87.577143,"[5739, 9068, 9260, 9769, 9713, 7984, 9771]"


In [22]:
total_dni = total[['NombreArchivo','DNI_x','dni_yolo','DNI_y','AcertividadDNI','acert_dni_mean','acerts_dni']]
total_dni = total_dni[total_dni.dni_yolo.apply(len)==total_dni.DNI_y.apply(len)]
total_dni['trues_est'] = total_dni['acerts_dni'].apply(lambda x : (np.array(x)<9500).sum())
total_dni['falses_est'] = total_dni['acerts_dni'].apply(lambda x : (np.array(x)>=9500).sum())
total_dni['trues_real'] = np_aciertos_from_str(total_dni['DNI_y'],total_dni['dni_yolo'])
total_dni['falses_real'] = total_dni['DNI_y'].apply(len) - total_dni['trues_real']
total_dni.head()

,NombreArchivo,DNI_x,dni_yolo,DNI_y,AcertividadDNI,acert_dni_mean,acerts_dni,trues_est,falses_est,trues_real,falses_real
2,MarzoCompras-0303072019121007_00025,04015560,04015000,04015880,60.25,88.40500,"[8678, 6025, 9630, 7339, 9627, 9851, 9654, 9920]",3,5,6,2
5,MarzoCompras-0303072019121014_00343,00270147,00740147,09740147,91.44,97.86125,"[9856, 9719, 9940, 9807, 9999, 9990, 9834, 9144]",1,7,7,1
7,MarzoCompras-0303072019121017_00459,08482229,00482219,08482279,74.83,91.77875,"[9957, 9401, 9468, 7483, 9954, 9463, 8462, 9235]",6,2,6,2
16,MarzoCompras-0303072019121023_00651,10758050,10798050,10748050,55.13,88.13500,"[9363, 9984, 9997, 5937, 9837, 9971, 9906, 5513]",3,5,7,1
17,MarzoCompras-0303072019121023_00652,70796052,10740057,10748050,72.20,90.69625,"[8373, 9939, 9975, 7227, 9900, 9989, 9934, 7220]",3,5,6,2


In [13]:
total_dni['trues_est'].sum(), total_dni['falses_est'].sum(), total_dni['trues_real'].sum(), total_dni['falses_real'].sum()

(5265, 7687, 9891, 3061)

In [122]:
for i in range(8000, 10000, 100):
    total_dni = total[['NombreArchivo','DNI_x','dni_yolo','DNI_y','AcertividadDNI','acert_dni_mean','acerts_dni']]
    total_dni = total_dni[total_dni.dni_yolo.apply(len)==total_dni.DNI_y.apply(len)]
    total_dni['trues_est'] = total_dni['acerts_dni'].apply(lambda x : (np.array(x)<i).sum())
    total_dni['falses_est'] = total_dni['acerts_dni'].apply(lambda x : (np.array(x)>=i).sum())
    total_dni['trues_real'] = np_aciertos_from_str(total_dni['DNI_y'],total_dni['dni_yolo'])
    total_dni['falses_real'] = total_dni['DNI_y'].apply(len) - total_dni['trues_real']
    te, fe, tr, fr = total_dni['trues_est'].sum(), total_dni['falses_est'].sum(), total_dni['trues_real'].sum(), total_dni['falses_real'].sum()
    # print('{} se tiene {} {} {} {}'.format(i/100, te, fe, tr, fr))
    print('{} se tiene {} {} {} {}'.format(i/100, round(te/(fr+tr),2), round(fe/(fr+tr),2), round(tr/(fr+tr),2), round(fr/(fr+tr),2)))

80.0 se tiene 0.14 0.86 0.76 0.24
81.0 se tiene 0.15 0.85 0.76 0.24
82.0 se tiene 0.16 0.84 0.76 0.24
83.0 se tiene 0.17 0.83 0.76 0.24
84.0 se tiene 0.18 0.82 0.76 0.24
85.0 se tiene 0.19 0.81 0.76 0.24
86.0 se tiene 0.21 0.79 0.76 0.24
87.0 se tiene 0.22 0.78 0.76 0.24
88.0 se tiene 0.24 0.76 0.76 0.24
89.0 se tiene 0.26 0.74 0.76 0.24
90.0 se tiene 0.27 0.73 0.76 0.24
91.0 se tiene 0.29 0.71 0.76 0.24
92.0 se tiene 0.31 0.69 0.76 0.24
93.0 se tiene 0.34 0.66 0.76 0.24
94.0 se tiene 0.37 0.63 0.76 0.24
95.0 se tiene 0.41 0.59 0.76 0.24
96.0 se tiene 0.45 0.55 0.76 0.24
97.0 se tiene 0.51 0.49 0.76 0.24
98.0 se tiene 0.59 0.41 0.76 0.24
99.0 se tiene 0.72 0.28 0.76 0.24


In [125]:
param_dni = total[['NombreArchivo','DNI_x','DNI_y','AcertividadDNI','acert_dni_mean']]

param_dni['trues_est'] = np.where(np.logical_and(param_dni['AcertividadDNI']>=84,param_dni.DNI_x.apply(len)==8), 1, 0)
param_dni['trues_real'] = np.where(param_dni['DNI_x']==param_dni['DNI_y'], 1, 0)
# param_dni['sta'] = np.where(param_dni['trues_est']==param_dni['trues_real'],'acierto',
                            # np.where(param_dni['trues_est']==1, 'Falso positivo', 'Verdadero negativo'))
    
# te, tr, tt = param_dni['trues_est'].sum(), param_dni['trues_real'].sum(), param_dni['trues_real'].values.size
param_dni

C:\Users\GYS\AppData\Local\conda\conda\envs\tfkeras\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\GYS\AppData\Local\conda\conda\envs\tfkeras\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,NombreArchivo,DNI_x,DNI_y,AcertividadDNI,acert_dni_mean,trues_est,trues_real
0,MarzoCompras-0303072019121007_00008,00622766,000672966,93.76,97.308750,1,0
1,MarzoCompras-0303072019121007_00010,0580806,09888016,0.00,77.671429,0,0
2,MarzoCompras-0303072019121007_00025,04015000,04015880,60.25,88.405000,0,0
3,MarzoCompras-0303072019121013_00265,031027,09310742,0.00,75.090000,0,0
4,MarzoCompras-0303072019121014_00331,082817622,08281360,0.00,84.056667,0,0
5,MarzoCompras-0303072019121014_00343,00740147,09740147,91.44,97.861250,1,0
6,MarzoCompras-0303072019121017_00450,9900157,09900151,0.00,94.557143,0,0
7,MarzoCompras-0303072019121017_00459,00482229,08482279,74.83,91.778750,0,0
8,MarzoCompras-0303072019121018_00497,452775,45279490,0.00,94.658333,0,0
9,MarzoCompras-0303072019121018_00501,0533314,05333314,0.00,87.577143,0,0


In [123]:
for i in range(70, 100, 1):
    param_dni = total[['NombreArchivo','DNI_x','dni_yolo','DNI_y','AcertividadDNI','acert_dni_mean']]
    # param_dni = param_dni[param_dni.dni_yolo.apply(len)==8]
    param_dni['trues_est'] = np.where(np.logical_and(param_dni['AcertividadDNI']>=i,param_dni.dni_yolo.apply(len)==8), 1, 0)
    param_dni['trues_real'] = np.where(param_dni['DNI_x']==param_dni['DNI_y'], 1, 0)
    
    param_dni['sta'] = np.where(param_dni['trues_est']==param_dni['trues_real'],'acierto',
                               np.where(param_dni['trues_est']==1, 'Falso positivo', 'Verdadero negativo'))
    
    te, tr, tt = param_dni['trues_est'].sum(), param_dni['trues_real'].sum(), param_dni['trues_real'].values.size
    # print('{} se tiene {} {} {}'.format(i, te, tr, tt))
    print('{} se tiene {} {}'.format(i, round(te/(tt),2), round(tr/(tt),2)))

70 se tiene 0.3 0.13
71 se tiene 0.28 0.13
72 se tiene 0.27 0.13
73 se tiene 0.26 0.13
74 se tiene 0.25 0.13
75 se tiene 0.24 0.13
76 se tiene 0.22 0.13
77 se tiene 0.21 0.13
78 se tiene 0.2 0.13
79 se tiene 0.19 0.13
80 se tiene 0.18 0.13
81 se tiene 0.17 0.13
82 se tiene 0.16 0.13
83 se tiene 0.15 0.13
84 se tiene 0.13 0.13
85 se tiene 0.12 0.13
86 se tiene 0.11 0.13
87 se tiene 0.1 0.13
88 se tiene 0.09 0.13
89 se tiene 0.07 0.13
90 se tiene 0.06 0.13
91 se tiene 0.05 0.13
92 se tiene 0.05 0.13
93 se tiene 0.04 0.13
94 se tiene 0.03 0.13
95 se tiene 0.02 0.13
96 se tiene 0.01 0.13
97 se tiene 0.01 0.13
98 se tiene 0.0 0.13
99 se tiene 0.0 0.13


C:\Users\GYS\AppData\Local\conda\conda\envs\tfkeras\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\GYS\AppData\Local\conda\conda\envs\tfkeras\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\GYS\AppData\Local\conda\conda\envs\tfkeras\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

In [134]:
param_dni = total[['NombreArchivo','DNI_x','dni_yolo','DNI_y','AcertividadDNI','acert_dni_mean']]
# param_dni = param_dni[param_dni.dni_yolo.apply(len)==8]
param_dni['trues_est'] = np.where(np.logical_and(param_dni['AcertividadDNI']>=97,param_dni.dni_yolo.apply(len)==8), 1, 0)
# param_dni['trues_est'] = np.where(np.logical_and(param_dni['acert_dni_mean']>=90,param_dni.dni_yolo.apply(len)==8), 1, 0)
param_dni['trues_real'] = np.where(param_dni['dni_yolo']==param_dni['DNI_y'], 1, 0)
# print('{} se tiene {} {}'.format(i, round(te/(tt),2), round(tr/(tt),2)))
pd.crosstab(param_dni['trues_est'],param_dni['trues_real'])

C:\Users\GYS\AppData\Local\conda\conda\envs\tfkeras\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\GYS\AppData\Local\conda\conda\envs\tfkeras\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


trues_real,0,1
trues_est,,
0,2806,324
1,13,16


In [135]:
norm = pd.crosstab(param_dni['trues_est'],param_dni['trues_real'],normalize=True)
np.round(norm * 100,2)

trues_real,0,1
trues_est,,
0,88.83,10.26
1,0.41,0.51


In [57]:
for i in range(80, 100, 1):
    param_dni = total[['NombreArchivo','DNI_x','dni_yolo','DNI_y','AcertividadDNI','acert_dni_mean']]
    # param_dni = param_dni[param_dni.dni_yolo.apply(len)==8]
    param_dni['trues_est'] = np.where(np.logical_and(param_dni['acert_dni_mean']>=i,param_dni.dni_yolo.apply(len)==8), 1, 0)
    param_dni['trues_real'] = np.where(param_dni['dni_yolo']==param_dni['DNI_y'], 1, 0)

    te, tr, tt = param_dni['trues_est'].sum(), param_dni['trues_real'].sum(), param_dni['trues_real'].values.size
    # print('{} se tiene {} {} {}'.format(i, te, tr, tt))
    print('{} se tiene {} {}'.format(i, round(te/(tt),2), round(tr/(tt),2)))

80 se tiene 0.51 0.11
81 se tiene 0.5 0.11
82 se tiene 0.5 0.11
83 se tiene 0.49 0.11
84 se tiene 0.48 0.11
85 se tiene 0.47 0.11
86 se tiene 0.45 0.11
87 se tiene 0.44 0.11
88 se tiene 0.42 0.11
89 se tiene 0.39 0.11
90 se tiene 0.36 0.11
91 se tiene 0.33 0.11
92 se tiene 0.28 0.11
93 se tiene 0.24 0.11
94 se tiene 0.2 0.11
95 se tiene 0.15 0.11
96 se tiene 0.11 0.11
97 se tiene 0.06 0.11
98 se tiene 0.03 0.11
99 se tiene 0.01 0.11


C:\Users\GYS\AppData\Local\conda\conda\envs\tfkeras\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\GYS\AppData\Local\conda\conda\envs\tfkeras\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [16]:
str_test = total[['LocalJsonOCR']].iloc[0,:].values[0]

In [27]:
json.loads(str_test)

{'dni_cnn': {'value': '00622926'},
 'telefono_cnn': {'value': '48236'},
 'dni_yolo': {'values': [0, 0, 6, 7, 2, 7, 6, 6],
  'scores': [9918, 9957, 9851, 9376, 9971, 9601, 9767, 9406],
  'boxes': [[17, 34, 60, 61],
   [16, 68, 55, 94],
   [17, 98, 53, 122],
   [14, 125, 52, 149],
   [11, 158, 53, 188],
   [1, 192, 57, 222],
   [3, 228, 53, 257],
   [6, 264, 50, 292]]},
 'telefono_yolo': {'values': [7, 0, 7, 2, 0],
  'scores': [8094, 9840, 9915, 9383, 8902],
  'boxes': [[6, 21, 48, 59],
   [6, 67, 49, 96],
   [0, 110, 42, 139],
   [0, 183, 44, 210],
   [5, 252, 48, 275]]}}

In [31]:
stringfy(json.loads(str_test)['dni_yolo']['values'])

'00672766'

### Azure

In [143]:
m3100_pro_azu = pd.read_csv('C:\\git\\cuponesWong\\CuponesWong\\notebooks_flow\\azure_result\\azure_result_muestra3100.csv', dtype={'DNI':str, 'Telefono':str})
m3100_pro_azu = m3100_pro_azu.fillna('')

In [144]:
m3100_pro_azu.head()

,NombreArchivo,DNI,AcertividadDNI,Telefono,AcertividadTelefono,NombreCompleto,AcertividadNombreCompleto,Direccion,AcertividadDireccion,Distrito,AcertividadDistrito,Correo,AcertividadCorreo,idCampania,idUsuario,idEstado
0,MarzoCompras-0303072019121007_00008,100672966,87.500000,98233140,85.0,rath,42.50,Av . Las Wauwtan # 246,86.875000,Chomilloy,85.000000,Karaujo710hohal,85.000000,3,1,2
1,MarzoCompras-0303072019121007_00010,09883016,85.000000,946498591,87.0,JUAN CARLOS SAlives AnDYA,86.25,AU. PERU N = 36 52,44.166667,SMP,90.000000,Mauricio2solSbHolmail.am,87.142857,3,1,2
2,MarzoCompras-0303072019121007_00025,04015880,87.500000,92848937,85.0,Jaime Roger Compos Benzospi,86.50,M Z H . LT. 12. HE. CAVIN DE HUONTAR.,86.904762,MILA EL SALVADOR.,86.666667,,0.000000,3,1,2
3,MarzoCompras-0303072019121013_00265,09310742,86.666667,985973705,85.0,Janet montero flores,86.00,Nelson Quia 415 A Opto 204,86.875000,Surco,85.000000,"Jalima9-yahoo,es",88.000000,3,1,2
4,MarzoCompras-0303072019121014_00331,08281361949352892,86.666667,949352892,87.0,howyo Morteness,42.50,? Joven nuevo news 14 7 + late I comite y,85.000000,Son Juan ducesamcris,85.000000,,0.000000,3,1,2


In [153]:
total_comp_azure.columns

Index(['NombreArchivo', 'DNI_x', 'AcertividadDNI', 'Telefono',
       'AcertividadTelefono', 'NombreCompleto', 'AcertividadNombreCompleto',
       'Direccion', 'AcertividadDireccion', 'Distrito', 'AcertividadDistrito',
       'Correo', 'AcertividadCorreo', 'idCampania', 'idUsuario', 'idEstado',
       'Unnamed: 0', 'nombre_archivo', 'DNI_y', 'telefono', 'Observación',
       'email', 'dni_lev_azure', 'tel_lev_azure'],
      dtype='object')

In [158]:
total_comp_azure = pd.merge(m3100_pro_azu, m3100_real, how='left', on='NombreArchivo')
total_comp_azure['dni_lev_azure'] = np_distance(total_comp_azure['DNI_x'],total_comp_azure['DNI_y'])
total_comp_azure['tel_lev_azure'] = np_distance(total_comp_azure['Telefono'],total_comp_azure['telefono'])
total_comp_azure['mail_lev_azure'] = np_distance(total_comp_azure['Correo'],total_comp_azure['email'])

In [159]:
total_comp_azure[['NombreArchivo','DNI_x','DNI_y','dni_lev_azure']].head()

,NombreArchivo,DNI_x,DNI_y,dni_lev_azure
0,MarzoCompras-0303072019121007_00008,100672966,000672966,1
1,MarzoCompras-0303072019121007_00010,09883016,09888016,1
2,MarzoCompras-0303072019121007_00025,04015880,04015880,0
3,MarzoCompras-0303072019121013_00265,09310742,09310742,0
4,MarzoCompras-0303072019121014_00331,08281361949352892,08281360,10


In [160]:
total_comp_azure[['NombreArchivo','Telefono','telefono','tel_lev_azure']].head()

,NombreArchivo,Telefono,telefono,tel_lev_azure
0,MarzoCompras-0303072019121007_00008,98233140,982331400,1
1,MarzoCompras-0303072019121007_00010,946498591,946498597,1
2,MarzoCompras-0303072019121007_00025,92848937,928489578,2
3,MarzoCompras-0303072019121013_00265,985973705,985973705,0
4,MarzoCompras-0303072019121014_00331,949352892,949357892,1


In [168]:
total_comp_azure[['NombreArchivo','Correo','email','mail_lev_azure']][total_comp_azure.mail_lev_azure==1]

,NombreArchivo,Correo,email,mail_lev_azure
2,MarzoCompras-0303072019121007_00025,,-,1
19,MarzoCompras-0303072019121025_00771,angela.reinoso@Upch.pe,angela.reinoso@upch.pe,1
20,MarzoCompras-0303072019121030_00931,,-,1
24,MarzoCompras-0303072019125012_00121,,-,1
41,MarzoCompras-0303072019132555_00124,Pattychic_14.7@hotmail.com,Pattychic_14_7@hotmail.com,1
45,MarzoCompras-0303072019132602_00385,alegna-19192@hotmail.com,alegna_19192@hotmail.com,1
50,MarzoCompras-0303072019132606_00553,,-,1
51,MarzoCompras-0303072019132606_00564,,-,1
59,MarzoCompras-0303072019132617_00962,,-,1
61,MarzoCompras-0303072019132618_01004,renzorojasperu456@gmailcom,renzorojasperu456@gmail.com,1


In [165]:
lev_dni_azure = total_comp_azure.groupby(['dni_lev_azure']).agg({'NombreArchivo':'count'})
lev_dni_azure['porcentaje'] = np.round(lev_dni_azure['NombreArchivo'] * 100 / lev_dni_azure['NombreArchivo'].sum() , 2)
lev_dni_azure['cuenta'] = lev_dni_azure['NombreArchivo']
lev_dni_azure[['cuenta','porcentaje']]

,cuenta,porcentaje
dni_lev_azure,,
0,1651,52.25
1,656,20.76
2,231,7.31
3,85,2.69
4,38,1.20
5,20,0.63
6,13,0.41
7,38,1.20
8,247,7.82


In [166]:
lev_tel_azure = total_comp_azure.groupby(['tel_lev_azure']).agg({'NombreArchivo':'count'})
lev_tel_azure['porcentaje'] = np.round(lev_tel_azure['NombreArchivo'] * 100 / lev_tel_azure['NombreArchivo'].sum() , 2)
lev_tel_azure['cuenta'] = lev_tel_azure['NombreArchivo']
lev_tel_azure[['cuenta','porcentaje']]

,cuenta,porcentaje
tel_lev_azure,,
0,1705,53.96
1,831,26.30
2,311,9.84
3,118,3.73
4,51,1.61
5,10,0.32
6,9,0.28
7,19,0.60
8,13,0.41


In [167]:
lev_mail_azure = total_comp_azure.groupby(['mail_lev_azure']).agg({'NombreArchivo':'count'})
lev_mail_azure['porcentaje'] = np.round(lev_mail_azure['NombreArchivo'] * 100 / lev_mail_azure['NombreArchivo'].sum() , 2)
lev_mail_azure['cuenta'] = lev_mail_azure['NombreArchivo']
lev_mail_azure[['cuenta','porcentaje']]

,cuenta,porcentaje
mail_lev_azure,,
0,1031,32.63
1,426,13.48
2,209,6.61
3,213,6.74
4,199,6.30
5,153,4.84
6,164,5.19
7,118,3.73
8,117,3.70
